In [ ]:
#第一段
from datetime import datetime, timedelta

start_date = "2018-07-01"
stop_date = "2018-12-31"                              #import datetime，並且定開始and結束日期

start = datetime.strptime(start_date, "%Y-%m-%d")
stop = datetime.strptime(stop_date, "%Y-%m-%d") 

dates = list()
while start <= stop:
    dates.append(start.strftime('%Y%m%d'))
    start = start + timedelta(days=1)                  #從起始日開始，將每天日期以20190306這種格式，加進list中，到結束日

In [ ]:
#第二段
import requests
from bs4 import BeautifulSoup as bs             #import request跟beautifulsuop，利用packages的功能才可開始抓取網路上的資料(吧

In [ ]:
#第三段
#是一個長長的function，有些部分還沒有看懂，但大概意思是把日期、標題、連結、內容，全部擷取進 data這個list
def process_document(document, date): 
    
    nodes = document.select('ul.list > li')
    data = list()

    for li in nodes:

        # check if is empty element
        if li.select_one('a') == None:
            continue

        # get link
        li_link = 'http://news.ltn.com.tw/' + li.select_one('a')['href']

        # request for document
        li_res = requests.get(li_link)
        li_doc = bs(li_res.text, 'lxml')

        # get date
        li_date = datetime.strptime(date, "%Y%m%d").strftime('%Y-%m-%d')

        #get title
        li_title = li.select_one('p').get_text()

        #get content
        li_content = ""
        for ele in li_doc.select('div.text > p'):
            if not 'appE1121' in ele.get('class', []):
                li_content += ele.get_text()

        # append new row
        data.append({
            'date' : li_date,
            'title': li_title,
            'link' : li_link,
            'content' : li_content,
            'tags' : []
        })
    return data

In [ ]:
#第四段
#設定目標網址(要擷取的某家新聞某個類型新聞)，一篇篇存進data這個list，然後把一個個名為data的list，存進all_data這個大list
cnt = 0
all_data = list()
for date in dates:
    print('start crawling :', date)
    res = requests.get('https://news.ltn.com.tw/list/newspaper/politics/' + date)
    doc = bs(res.text, 'lxml')
    data = process_document(doc, date)
    all_data += data 
    

In [ ]:
#第五段
all_data[0:5]                 #抓前5個出來檢查結果是否有誤

In [ ]:
#第六段
#存pkl檔，用with語法開啟一個f，名為'data/liberty_times.pkl'，將all_data存進f
import pickle 

with open('data/liberty_times.pkl', 'wb') as f:
    pickle.dump(all_data, f)

In [ ]:
#第七段
#用pandas將資料做成表格
import pandas as pd
pd.DataFrame(all_data)[['date', 'title', 'link', 'content', 'tags']].head() 